In [1]:
# pip install langchain_core langchain_community sentence_transformers weaviate_client

In [2]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Weaviate
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [3]:
import os
import weaviate
import ssl

os.environ['CURL_CA_BUNDLE'] = ''  # disables cert validation
ssl._create_default_https_context = ssl._create_unverified_context

from datasets import load_dataset, load_from_disk
from transformers import (
    AutoModel,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    pipeline,
)

# dataset_name = "openai/summarize_from_feedback"
# dataset = load_dataset(dataset_name, name='comparisons')

In [4]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"  # "sentence-transformers/paraphrase-MiniLM-L6-v2"  #
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    # force_download=True,
)

model_id = "microsoft/phi-1_5"  # "microsoft/Phi-3-mini-4k-instruct" #"google/flan-t5-xl" # "/Users/gtoth/Documents/models/gemma-2b-it" # #  # #"microsoft/phi-1_5" #

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)  # AutoModelForSeq2SeqLM
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    device="mps",
)
llm = HuggingFacePipeline(pipeline=pipe)

/Users/owner/opt/anaconda3/envs/llm-diss/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/owner/opt/anaconda3/envs/llm-diss/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [5]:
client = weaviate.Client(
    embedded_options=weaviate.embedded.EmbeddedOptions(),
)

/Users/owner/opt/anaconda3/envs/llm-diss/lib/python3.11/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


Started /Users/owner/.cache/weaviate-embedded: process ID 11346


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-06-25T19:25:53+01:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-06-25T19:25:53+01:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-06-25T19:25:53+01:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-06-25T19:25:54+01:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-06-25T19:25:54+01:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

In [6]:
docs = [
    Document(
        page_content="""Categories:
0. Unknown category.
1. Intents related to flights.
2. Intents related to hotel booking.
Intents:
0.0 Unknown intent.
1.0. The user wants to check the status of a flight.
1.1. The user wants to book a flight.

Utterance: I would like to make a flight booking.
Intent: 1.1""",
        metadata={"example": "basic"},
    ),
]
vectorstore = Weaviate.from_documents(
    [],
    embeddings,
    weaviate_url="http://127.0.0.1:8079",  # "http://127.0.0.1:8079"
)
retriever = vectorstore.as_retriever(k=2)

/Users/owner/opt/anaconda3/envs/llm-diss/lib/python3.11/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(
{"level":"info","msg":"Created shard langchain_d0603ed15bfe46af86a1781748edbfb2_LqzV9Z2FkQ6I in 2.641044ms","time":"2024-06-25T19:25:54+01:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-06-25T19:25:54+01:00","took":302758}


In [22]:
retriever = vectorstore.as_retriever(k=2)  # search_type="hybrid", alpha=0.5, beta=0.99
# docsearch.as_retriever(
#                 search_type="similarity_score_threshold",
#                 search_kwargs={'score_threshold': 0.8}
#             )
# from langchain_community.retrievers import WeaviateHybridSearchRetriever

# retriever = WeaviateHybridSearchRetriever(
#     client=client,
#     index_name="LangChain",
#     text_key="text",
#     attributes=[],
#     # create_schema_if_missing=True,
#     alpha=0.5,
#     k=2
# )
#
# def get_content_str_from_docs(retrieved_documents: List[Document]) -> str:
#     relevant_content = (
#         [doc.page_content for doc in retrieved_documents] if len(retrieved_documents) >= 1 else []
#     )
#     return '\n'.join(relevant_content)
# example_retriever = vectorstore.as_retriever(
#                 search_kwargs={
#                     "k": 1,
#                     "where_filter": {"path": ["type"], "operator": "Equal", "valueString": "example"},
#                 }
#             )
#
# intent_retriever = vectorstore.as_retriever(
#     search_type=search_kwargs["search_type"],
#     search_kwargs={
#         "k": search_kwargs["k"],
#         "where_filter": {"path": ["type"], "operator": "Equal", "valueString": "intent"},
#     },
# )
# # prompt_assembly_chain = {"utterance": itemgetter("utterance"),
# #                                  "language_code": itemgetter("language_code"),
# #                                  "example": itemgetter("utterance")
# #                                             | example_retriever
# #                                             | RunnableLambda(get_content_str_from_docs),
# #                                  # "categories": itemgetter("utterance") | category_retriever,
# #                                  "intents": itemgetter("utterance")
# #                                             | intent_retriever
# #                                             | RunnableLambda(get_content_str_from_docs),
# #                                             # itemgetter("intents_str"),
# #                                  } | prompt
# #
# # correct_example = [Document(
# #                             page_content=f"""Intents:
# # {retrieved_intents}
# # Intent: {response['intent']}""",
# #                             metadata={"type": "example", "origin": "collected"},
# #                         )]
# #                         # print(correct_example)
# #                         example_retriever.add_documents(documents=correct_example)

In [23]:
template = """<bos>You're an intent classifier for language: '{language_code}'.
Analyze the utterance and choose the most relevant intent and category from the list that best matches the utterance.
    - Use only the provided intents and categories.
    - Return "0.0" if no intents and category match.
    - Return only the number of the most relevant intent and nothing else.
    
<example>
{example}
</example>
    
Categories:
{categories}
Intents:
{intents}

Utterance: {utterance}
Intent:"""
# Instantiation using from_template (recommended)
prompt = PromptTemplate.from_template(template)
# prompt.format(foo="bar")

# Instantiation using initializer
# prompt = PromptTemplate(input_variables=["foo"], template="Say {foo}")

In [24]:
import json
from pprint import pprint
from typing import List

from pydantic import BaseModel, Field


json_path = '/Users/gtoth/PycharmProjects/grok/grok_intent_evaluation/cortex_bank_bot_definition_1.json'

with open(json_path) as f:
    intents_config = json.load(f)


class Intent(BaseModel):
    name: str
    description: str


class Category(BaseModel):
    name: str
    description: str
    intents: List[Intent]


class Domain(BaseModel):
    name: str
    description: str
    categories: List[Category]

In [25]:
# intents_config = intents_config[0]
# pprint(intents_config[0])
categories = Domain.model_validate(intents_config[0]).categories
# pprint(categories)
categories_list = ["0. None of these categories match"]

intents_config
for index, category in enumerate(categories, start=1):
    categories_list.append(f"{index}. {category.description}")

categories_str = "\n".join(categories_list)

intent_list = ["0.0. None of the other intents match, fallback to this intent"]
index_to_intent_map = {"0.0": "Unknown"}
index_to_category_map = {"0": "Unknown"}
for cat_index, category in enumerate(categories, start=1):
    index_to_category_map[f"{cat_index}"] = category.name
    for intent_index, intent in enumerate(category.intents, start=1):
        index_to_intent_map[f"{cat_index}.{intent_index}"] = intent.name
        intent_list.append(f"{cat_index}.{intent_index}. {intent.description}")

intents_str = "\n".join(intent_list)

print(categories_str)
print(intents_str)

0. None of these categories match
1. Handles tasks related to bank account management.
2. Provides information and assistance related to various financial services.
3. Provides assistance and resolves issues related to the bank's services.
0.0. None of the other intents match, fallback to this intent
1.1. The user is looking to open a new bank account.
1.2. The user has a query related to logging into their existing bank account.
1.3. The user wants to check the amount of money they have in their bank account.
1.4. The user is looking to close their existing bank account.
1.5. The user is requesting a copy of their bank account statement.
1.6. The user has a question or inquiry about a specific transaction on their bank account.
1.7. The user wants to update their personal or contact information on their bank account.
2.1. The user wants to apply for a new credit card from the bank.
2.2. The user wants to exchange one currency for another.
2.3. The user is applying for a loan, such as 

In [28]:
chain = (
    {
        "example": itemgetter("utterance") | retriever,  # gets k=2 documents
        "utterance": itemgetter("utterance"),
        "language_code": itemgetter("language_code"),
        "categories": itemgetter("categories"),
        "intents": itemgetter("intents"),
    }
    | prompt
    | llm.bind(stop=["\n\n", "Response", '. '])
    | StrOutputParser()
)  # .bind(stop=["\n\n", "Response"])

# critique_chain = call_llm_to_do_cot | call_llm_to_label | if_answer is good then done else | critique | label_again
# expel_chain = get_insights | get_good_or_bad_examples | call_llm_to_do_cot | call_llm_to_label | if_answer is good then done else | create_insights | label_again

In [30]:
# response = chain.invoke({"question": "where do a psychologist and detective get lost?"})
# print(response)
response = chain.invoke(
    {
        "utterance": "need to see all my deposits from the last six months",
        "language_code": "en-US",
        "categories": categories_str,
        "intents": intents_str,
    }
).strip()

print(response)

detected_intent = index_to_intent_map[response]
# get expected intent
# if match then save example to DB.
# could also add intents into DB and filter based on example.

'3.5'

In [34]:
# chain.batch([#{"question": "Where do 3 men walk into?"},
#              {"question": "Where do a psychologist and detective get lost?"},
#              {"question": "What did scientist bring back?"}],
#             config={"max_concurrency": 5})

In [13]:
# check how custom function can be written to branch execution below.
# retriever.add_documents(docs) # with see docs above

In [29]:
other_chain = (
    {
        "context": itemgetter("question") | retriever,  # gets k=2 documents
        "question": itemgetter("question"),
    }
    | prompt
    | llm.bind(logprobs=True)
)  # | StrOutputParser()
msg = other_chain.invoke(
    {"question": "where do a psychologist and detective get lost?"}
)
print(msg)


The provided text does not specify where a psychologist and detective get lost, so I am unable to answer this question from the provided context.


In [34]:
tokenizer.pad_token = tokenizer.eos_token
target_token_id = tokenizer(
    ['2', '1'],
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)
type(target_token_id)

transformers.tokenization_utils_base.BatchEncoding

### END